# ODMD on pairing Hamiltonian

We test the ODMD algorithm proposed in [arXiv:2306.01858](https://arxiv.org/abs/2306.01858) on the pairing Hamiltonian. 

Let us define some utils...

In [130]:
Norb = 4
Nocc = 2
gval = 0.33  

In [131]:
import numpy as np
import scipy
import itertools
from itertools import combinations
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_aer.primitives import SamplerV2
from qiskit.circuit.library import PauliEvolutionGate, PauliGate
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.synthesis import SuzukiTrotter
from qiskit.visualization import plot_histogram
import seaborn as sns

class PairingHamiltonian:
    def __init__(self, Norb, Nocc, gval, delta_eps=1.0):
        self.Norb = Norb
        self.Nocc = Nocc
        self.delta_eps = delta_eps
        self.gval = gval
        self.basis = self.make_basis()
        self.epsilon = self.eval_epsilon()
        self.Hmat = self.eval_Hmat()

    def make_basis(self):
        self.basis = []
        for occ in combinations(range(self.Norb), self.Nocc):
            self.basis.append(occ)

        return self.basis
    
    def eval_epsilon(self):
        self.epsilon = [ 2 * i * self.delta_eps for i in range(self.Norb) ]
        return self.epsilon
    
    def eval_Hmat(self):
        dim = len(self.basis)
        self.Hmat = np.zeros((dim, dim))
        for bra_idx, bra in enumerate(self.basis):
            for ket_idx, ket in enumerate(self.basis):
                # Hamming distance
                diff = [ i for i in bra if i not in ket ]
                same = [ i for i in bra if i in ket ]
                # for SPE term
                if bra_idx == ket_idx:
                    self.Hmat[bra_idx, ket_idx] += np.sum( [self.epsilon[i] for i in same])
                    self.Hmat[bra_idx, ket_idx] += - self.gval * len(same) 
                # for pairing term
                if len(diff) == 1:
                    self.Hmat[bra_idx, ket_idx] = - self.gval

        return self.Hmat

def tuple_to_bitstring(tup, Norb, rev=True):
    bitint = 0
    for i in tup:
        bitint += 2**i
    if rev:
        bitstring = "|"+format(bitint, f'0{Norb}b')[::-1]+">"
    else:
        bitstring = "|"+format(bitint, f'0{Norb}b')+">"        
    return bitstring

def cG1(circ, c_qubit, i, j, theta):
    theta_4 = theta / 4 
    circ.cx(i,j)
    circ.ry(theta_4, i)
    circ.cx(j,i)
    circ.ry(-theta_4, i)
    circ.cx(c_qubit, i)
    circ.ry(theta_4, i)
    circ.cx(j,i)
    circ.ry(-theta_4, i)
    circ.cx(c_qubit, i)
    circ.cx(i,j)

def cG1_gate(theta):
    circ = QuantumCircuit(2)
    G(circ, 0, 1, theta)
    circ.name = "cG1"   
    circ = circ.to_gate()
    circ = circ.control(1) 
    return circ

def G(circ, i, j, theta):
    theta_2 = theta / 2 
    circ.cx(i,j)
    circ.ry(theta_2, i)
    circ.cx(j,i)
    circ.ry(-theta_2, i)
    circ.cx(j,i)
    circ.cx(i,j)  

def G_gate(theta):
    circ = QuantumCircuit(2)
    G(circ, 0, 1, theta)
    circ.name = "G"    
    return circ.to_gate()

def get_idx_ancilla_in_string(n_qubit, ancilla, Qiskit_ordering):
    idx_ancilla = None
    if ancilla != None:
        if Qiskit_ordering:
            idx_ancilla = n_qubit-1 - ancilla
        else:
            idx_ancilla = ancilla
    return idx_ancilla

def expec_Z(counts, i, Qiskit_ordering=True, target_qubits=[], ancilla_qubit=None):
    Zexp = Zexp_p0 = Zexp_p1 = 0.0
    n_shot = sum(counts.values())
    n_qubit = len(list(counts.keys())[0])
    idx_ancilla = get_idx_ancilla_in_string(n_qubit, ancilla_qubit, Qiskit_ordering)
    for bitstr, count in counts.items():
        if ancilla_qubit != None and target_qubits != []:
            bitstr_target = ''.join([bitstr[k] for k in range(n_qubit) if k != idx_ancilla])
        else:
            bitstr_target = bitstr

        if Qiskit_ordering:
            idx_register = -1 - i
        else:
            idx_register = i
        bit = int(bitstr_target[idx_register])
        tmp = (1 - 2 * bit) * count
        Zexp += tmp
        if ancilla_qubit != None:
            if int(bitstr[idx_ancilla]) == 0:
                Zexp_p0 += tmp
            else:
                Zexp_p1 += tmp
    Zexp /= n_shot
    Zexp_p0 /= n_shot
    Zexp_p1 /= n_shot
    return Zexp, Zexp_p0, Zexp_p1

def expec_ZZ(counts, i, j, Qiskit_ordering=True, target_qubits=[], ancilla_qubit=None):
    ZZexp = ZZexp_p0 = ZZexp_p1 = 0.0
    n_shot = sum(counts.values())
    n_qubit = len(list(counts.keys())[0])
    idx_ancilla = get_idx_ancilla_in_string(n_qubit, ancilla_qubit, Qiskit_ordering)

    for bitstr, count in counts.items():
        if ancilla_qubit != None and target_qubits != []:
            bitstr_target = ''.join([bitstr[k] for k in range(n_qubit) if k != idx_ancilla])
            #print(bitstr ,"=>?? ", bitstr_target)
        else:
            bitstr_target = bitstr

        if Qiskit_ordering:
            idx_register_i = -1 - i
            idx_register_j = -1 - j
        else:
            idx_register_i = i
            idx_register_j = j
        bit_i = int(bitstr_target[idx_register_i])
        bit_j = int(bitstr_target[idx_register_j])
        tmp = (1 - 2 * bit_i) * (1 - 2 * bit_j) * count
        ZZexp += tmp
        if ancilla_qubit != None:
            if int(bitstr[idx_ancilla]) == 0:
                ZZexp_p0 += tmp
            else:
                ZZexp_p1 += tmp
    ZZexp /= n_shot; ZZexp_p0 /= n_shot; ZZexp_p1 /= n_shot
    return ZZexp, ZZexp_p0, ZZexp_p1

def additional_qc(qc_in, pauli_str, register_target, Qiskit_order=True):
    numX = pauli_str.count("X")
    numY = pauli_str.count("Y")
    numZ = pauli_str.count("Z")
    if (numX == numY == numZ == 0) or (numZ == 1 and numX == numY == 0): #I or Z, no need to add gate
        return None
    elif numX == 2 and numY == numZ == 0: # X_i X_j
        if Qiskit_order:
            i, j = -1 - pauli_str.index("X"), -1 - pauli_str.rindex("X")
        else:
            i, j = pauli_str.index("X"), pauli_str.rindex("X")
        qc_in.h(register_target[i])
        qc_in.h(register_target[j])
    elif numY == 2 and numX == numZ == 0: # Y_i Y_j
        if Qiskit_order:
            i, j = -1 - pauli_str.index("Y"), -1 - pauli_str.rindex("Y")
        else:
            i, j = pauli_str.index("Y"), pauli_str.rindex("Y")
        qc_in.sdg(register_target[i]); qc_in.h(register_target[i])
        qc_in.sdg(register_target[j]); qc_in.h(register_target[j])
    else: 
        raise ValueError("This must not happen:", pauli_str)

def get_idx_to_measure(pauli_str, Qiskit_order=True):
    numX = pauli_str.count("X")
    numY = pauli_str.count("Y")
    numZ = pauli_str.count("Z")
    if numX == numY == numZ == 0: #I
        return [ ]
    elif numZ == 1 and numX == numY == 0: # Z
        if Qiskit_order:
            return [-1 - pauli_str.index("Z")]
        else:
            return [pauli_str.index("Z")]
    elif numX == 2 and numY == numZ == 0: # X_i X_j
        if Qiskit_order:
            return [-1 - pauli_str.index("X"), -1 - pauli_str.rindex("X")]
        else:
            return [pauli_str.index("X"), pauli_str.rindex("X") ]
    elif numY == 2 and numX == numZ == 0: # Y_i Y_j
        if Qiskit_order:
            return [-1 - pauli_str.index("Y"), -1 - pauli_str.rindex("Y")]
        else:
            return [pauli_str.index("Y"), pauli_str.rindex("Y")]
    else: 
        raise ValueError("This must not happen:", pauli_str)


params_exact = np.array([-0.48104276, -1.03976498, -0.98963981, -1.18481738, -0.54832984])

Eshift = 0.0

def get_PairingHamiltonian(Norb, Nocc, gval, Eshift=0.0):
    Hamil = PairingHamiltonian(Norb, Nocc, gval)
    SPEs = Hamil.epsilon
    pauli_list = [ ]
    obs = [ ]
    coeffs = [ ]

    # I term
    coeff = 0.0
    op = "I" * Norb
    for i in range(Norb):
        coeff += 0.5 * ( SPEs[i] - Hamil.gval ) 
    obs += [op]
    coeff -= Eshift
    coeffs += [coeff]

    # -Zp term
    for i in range(Norb):
        op = "I" * Norb
        op = op[:i] + "Z" + op[i+1:]
        coeff = -0.5 * ( SPEs[i] - Hamil.gval )

        op = op[::-1]
        obs += [op]
        coeffs += [coeff]
    # XX+YY term
    for i in range(Hamil.Norb):
        for j in range(i+1, Hamil.Norb):
            factor = - Hamil.gval / 2
            op = "I" * Norb
            op = op[:i] + "X" + op[i+1:j] + "X" + op[j+1:]
            op = op[::-1]
            obs += [op]
            coeffs += [ factor ]
            op = "I" * Norb
            op = op[::-1]
            op = op[:i] + "Y" + op[i+1:j] + "Y" + op[j+1:]
            obs += [op]
            coeffs += [ factor ]

    return SparsePauliOp(obs, coeffs)

def ansatz(params, method="FCI"):
    qc = QuantumCircuit(Norb)
    # HF
    for i in range(Nocc):
        qc.x(i)
    # Additional gates
    if method == "FCI":
        if Norb != 4 or Nocc != 2:
            print("You are using parameters to reproduce the FCI wavefunction for Norb=4 and Nocc=2")
            print("Please make sure whether this is correct for your case!")
        qc.append(G_gate(params[0]), [1, 2])
        qc.append(G_gate(params[1]), [2, 3])
        qc.append(cG1_gate(params[2]), [2, 0, 1])
        qc.append(cG1_gate(params[3]), [3, 0, 1])
        qc.append(cG1_gate(params[4]), [3, 1, 2])        
    return qc

hamiltonian_pauli_ops = get_PairingHamiltonian(Norb, Nocc, gval, Eshift)
Hamil = PairingHamiltonian(Norb, Nocc, gval)
evals, evecs = np.linalg.eigh(Hamil.Hmat)
evals = np.linalg.eigvalsh(Hamil.Hmat)
Egs_exact = evals[0]
E_HF = Hamil.Hmat[0,0]

print("# of orbitals: ", Norb, " # of particles: ", Nocc)
print("basis:", Hamil.basis)
print([tuple_to_bitstring(tup, Norb) for tup in Hamil.basis])
#print("eps: ", Hamil.epsilon)
#print("Hmat: ", Hamil.Hmat)
print("evals: ", evals)
print("Egs_exact: ", Egs_exact, " E_HF", E_HF)
#print("gs evec", evecs[:,0])
#print("gs amp.", evecs[:,0]**2)

# of orbitals:  4  # of particles:  2
basis: [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
['|1100>', '|1010>', '|1001>', '|0110>', '|0101>', '|0011>']
evals:  [1.18985184 3.29649666 5.34       5.34       7.42853393 9.44511758]
Egs_exact:  1.1898518351360725  E_HF 1.3399999999999999


In [132]:
def make_U_and_cU(i, delta_t, trotter_steps, hamiltonian_op, ancilla_qubits, target_qubits, Uprep):
    Ntar = len(target_qubits)
    time = i * delta_t
    circuit_U = QuantumCircuit(Ntar)
    expiHt = PauliEvolutionGate(hamiltonian_op, time, synthesis=SuzukiTrotter(order=1,reps=trotter_steps))
    circuit_U.append(expiHt, range(Ntar))
    qc_U = circuit_U.decompose().decompose()
    if using_noisy_simulator:
        qc_U = transpile(qc_U, backend=backend, optimization_level=2)
    
    qc_cU = QuantumCircuit(Ntar)
    qc_cU.append(Uprep, range(Ntar))
    qc_cU.append(expiHt, range(Ntar))
    qc_cU = qc_cU.decompose().decompose()
    qc_cU = qc_cU.to_gate().control(1)
    if using_noisy_simulator:
        qc_cU = transpile(qc_cU, backend=backend, optimization_level=2)
    return qc_U, qc_cU

def make_overlap_qc(Ntar, gate_cUi, gate_cUj, ancilla_qubits, target_qubits, using_statevector):
    qc_re = QuantumCircuit(1+Ntar, 1)
    qc_re.h(0)
    qc_re.append(gate_cUi, ancilla_qubits + target_qubits)
    qc_re.x(0)
    qc_re.append(gate_cUj, ancilla_qubits+target_qubits)
    qc_re.h(0)
    if not using_statevector:
        qc_re.measure(0,0)
    qc_re = qc_re.decompose(reps=4)
    if using_noisy_simulator:
        qc_re = transpile(qc_re, backend=backend, optimization_level=2)

    # Overlap, Im part 
    qc_im = QuantumCircuit(1+Ntar,1)
    qc_im.h(0)
    qc_im.append(gate_cUi, ancilla_qubits + target_qubits)
    qc_im.x(0)
    qc_im.append(gate_cUj, ancilla_qubits + target_qubits)
    qc_im.sdg(0)
    qc_im.h(0)
    if not using_statevector:
        qc_im.measure(0,0)
    qc_im = qc_im.decompose(reps=4)
    if using_noisy_simulator:
        qc_im = transpile(qc_im, backend=backend, optimization_level=2)

    return qc_re, qc_im

def measure_overlap(num_shot, Ntar, gate_cUi, gate_cUj, ancilla_qubits, target_qubits, sampler, using_statevector):
    qc_re, qc_im = make_overlap_qc(Ntar, gate_cUi, gate_cUj, ancilla_qubits, target_qubits, using_statevector)

    if using_statevector:
        results = [ Statevector.from_instruction(qc).probabilities_dict() for qc in [qc_re, qc_im]]
        prob_Re = results[0]
        prob_Im = results[1] 
    else:
        if using_noisy_simulator:
            result = sampler.run([qc_re,qc_im], shots=num_shot).result()
            result_Re = result.results[0].data.counts
            nshot = sum(result_Re.values())
            p0 = result_Re.get('0x1', 0) / nshot
            p1 = result_Re.get('0x0', 0) / nshot
            ReN = p0 - p1

            result_Im = result.results[1].data.counts 
            nshot = sum(result_Im.values())
            p0 = result_Im.get('0x1', 0) / nshot
            p1 = result_Im.get('0x0', 0) / nshot
            ImN = p0 - p1
        else:
            job = sampler.run([qc_re, qc_im], shots=num_shot)
            results  = job.result()
            prob_Re = results[0].data.c.get_counts()
            prob_Im = results[1].data.c.get_counts()

            p0 = np.sum( [ count for bitstr, count in prob_Re.items() if bitstr[-1] == '0' ] ) / np.sum(list(prob_Re.values()))
            p1 = np.sum( [ count for bitstr, count in prob_Re.items() if bitstr[-1] == '1' ] ) / np.sum(list(prob_Re.values()))
            ReN = p0 - p1

            p0 = np.sum( [ count for bitstr, count in prob_Im.items() if bitstr[-1] == '0' ] ) / np.sum(list(prob_Im.values()))
            p1 = np.sum( [ count for bitstr, count in prob_Im.items() if bitstr[-1] == '1' ] ) / np.sum(list(prob_Im.values()))
            ImN = p0 - p1

    U_ij = ReN + 1j * ImN
    return U_ij

def make_cU(Uprep, Ui, Ntar):
    circuit_cUi = QuantumCircuit(Ntar)
    circuit_cUi.append(Uprep, range(Ntar))
    circuit_cUi.append(Ui, range(Ntar))
    circuit_cUi = circuit_cUi.decompose(reps=4)
    return circuit_cUi.to_gate().control(1)

def make_nonD_H_qc(op_string, Ntar, ancilla_qubits, target_qubits, gate_cUi, gate_cUj, qcs_re, qcs_im, using_statevector):
    # real part
    qc_reH_nonD = QuantumCircuit(1+Ntar)
    qc_reH_nonD.h(0)
    qc_reH_nonD.append(gate_cUi, ancilla_qubits + target_qubits)
    qc_reH_nonD.x(0)
    qc_reH_nonD.append(gate_cUj, ancilla_qubits + target_qubits)
    qc_reH_nonD.h(0)
    additional_qc(qc_reH_nonD, op_string, target_qubits)
    if not using_statevector:
        qc_reH_nonD.measure_all()
    if using_noisy_simulator:
        qc_reH_nonD = transpile(qc_reH_nonD, backend=backend, optimization_level=2)
    else:
        qc_reH = qc_reH_nonD.decompose().decompose()
    qcs_re.append(qc_reH)

    # imaginary part
    qc_imH_nonD = QuantumCircuit(1+Ntar)
    qc_imH_nonD.h(0)
    qc_imH_nonD.append(gate_cUi, ancilla_qubits + target_qubits)
    qc_imH_nonD.x(0)
    qc_imH_nonD.append(gate_cUj, ancilla_qubits + target_qubits)
    qc_imH_nonD.sdg(0)
    qc_imH_nonD.h(0)
    additional_qc(qc_imH_nonD, op_string, target_qubits)
    if not using_statevector:
        qc_imH_nonD.measure_all()
    if using_noisy_simulator:
        qc_imH = transpile(qc_imH_nonD, backend=backend, optimization_level=2)
    else:
        qc_imH = qc_imH_nonD.decompose().decompose()
    qcs_im.append(qc_imH)

    return None

def QuantumKrylov(Uprep: QuantumCircuit, # circuit to prepare a reference state
                 hamiltonian_op: SparsePauliOp, # Hamiltonian operator
                 delta_t=0.01,
                 max_iterations=10,
                 trotter_steps=1, 
                 num_shot=10**5, 
                 ancilla_qubits=[0], target_qubits=list(range(1,Norb+1)),
                 using_statevector=False
                 ):
    Ntar = len(target_qubits)
    N = np.zeros((max_iterations, max_iterations), dtype=np.complex128)
    H = np.zeros((max_iterations, max_iterations), dtype=np.complex128)
    ws = []
    Unitaries = [ ]
    for it in range(max_iterations):
        print("iteration: ", it)
        ## make controlled U = exp(-iHδt * it)
        Ui = PauliEvolutionGate(hamiltonian_op, it*delta_t, synthesis=SuzukiTrotter(order=1,reps=trotter_steps))
        gate_cUi = make_cU(Uprep, Ui, Ntar)
        Unitaries.append(gate_cUi)
        N[it, it] = 1.0
        ## evaluate overlap to previous states
        for j in range(it-1, -1, -1):
            gate_cUj = Unitaries[j]
            U_ij = measure_overlap(num_shot, Ntar, gate_cUi, gate_cUj, ancilla_qubits, target_qubits, sampler, using_statevector)
            N[it, j] = U_ij
            N[j, it] = np.conj(U_ij)
        ### evaluate H_ii no need ancilla qubit
        qcs = []  
        for idx_H in range(len(Hamil_paulis)):
            op_string = Hamil_paulis[idx_H].to_label()
            idx_relevant = get_idx_to_measure(op_string)
            qc_reH_D = QuantumCircuit(Ntar)
            qc_reH_D.append(Uprep, range(Ntar))
            qc_reH_D.append(Ui, range(Ntar))
            additional_qc(qc_reH_D, op_string, range(Ntar))
            if not(using_statevector):
                qc_reH_D.measure_all()
            qc_reH_D = qc_reH_D.decompose().decompose()
            qcs.append(qc_reH_D)
        if using_statevector:
            results = [ Statevector.from_instruction(qc).probabilities_dict() for qc in qcs]
        else:
            job = sampler.run(qcs, shots=num_shot)
            results  = job.result()

        Hsum = 0.0
        for idx_H in range(len(Hamil_paulis)):
            op_string = Hamil_paulis[idx_H].to_label()
            idx_relevant = get_idx_to_measure(op_string)
            if using_statevector:
                res = results[idx_H]
            else:
                res = results[idx_H].data.meas.get_counts()
            if len(idx_relevant) == 0: # I term
                expval = 1.0
            elif len(idx_relevant) == 1: 
                expval, dummy, dummy_ = expec_Z(res, idx_relevant[0])
            elif len(idx_relevant) == 2: 
                expval, dummy, dummy_ = expec_ZZ(res, idx_relevant[0], idx_relevant[1])

            Hsum += Hamil_coeffs[idx_H] * expval
            ##print("operator: ", op_string, "coeff: ", Hamil_coeffs[idx_H], "exp. val: ",  expval)
        #print("H[it, it]", it, Hsum)
        H[it, it] = Hsum

        ### evaluate H_ij ancilla qubit is needed
        for j in range(it-1, -1, -1):
            gate_cUj = Unitaries[j]
            qcs_re = []
            qcs_im = []
            for idx_H in range(len(Hamil_paulis)):
                op_string = Hamil_paulis[idx_H].to_label()
                make_nonD_H_qc(op_string, Ntar, ancilla_qubits, target_qubits, gate_cUi, gate_cUj, qcs_re, qcs_im, using_statevector)
            # Re part
            if using_statevector:
                results = [ Statevector.from_instruction(qc).probabilities_dict() for qc in qcs_re]
            else:
                job = sampler.run(qcs_re, shots=num_shot)
                results  = job.result()
            Re_H_ij = 0.0
            for idx_H in range(len(Hamil_paulis)):
                op_string = Hamil_paulis[idx_H].to_label()
                idx_relevant = get_idx_to_measure(op_string)
                if using_statevector:
                    res = results[idx_H]
                else:
                    res = results[idx_H].data.meas.get_counts()
                if len(idx_relevant) == 0: # I term
                    p0 = np.sum( [ count for bitstr, count in res.items() if bitstr[-1] == '0' ] ) / np.sum(list(res.values()))
                    p1 = np.sum( [ count for bitstr, count in res.items() if bitstr[-1] == '1' ] ) / np.sum(list(res.values()))
                elif len(idx_relevant) == 1: 
                    dummy_e, p0, p1 = expec_Z(res, idx_relevant[0], target_qubits=range(len(target_qubits)), ancilla_qubit=0)
                elif len(idx_relevant) == 2: 
                    dummy_e, p0, p1 = expec_ZZ(res, idx_relevant[0], idx_relevant[1], target_qubits=range(len(target_qubits)), ancilla_qubit=0)
                expval = p0 - p1
                Re_H_ij += Hamil_coeffs[idx_H] * expval
                #print("operator: ", op_string, "coeff: ", Hamil_coeffs[idx_H], "exp. val: ",  expval)
            #print("Re H_{ij}", Re_H_ij)

            # Im part
            if using_statevector:
                results = [ Statevector.from_instruction(qc).probabilities_dict() for qc in qcs_im]
            else:
                job = sampler.run(qcs_im, shots=num_shot)
                results  = job.result() 
            Im_H_ij = 0.0
            for idx_H in range(len(Hamil_paulis)):
                op_string = Hamil_paulis[idx_H].to_label()
                idx_relevant = get_idx_to_measure(op_string)
                if using_statevector:
                    res = results[idx_H]
                else:
                    res = results[idx_H].data.meas.get_counts()   
                if len(idx_relevant) == 0:
                    p0 = np.sum( [ count for bitstr, count in res.items() if bitstr[-1] == '0' ] ) / np.sum(list(res.values()))
                    p1 = np.sum( [ count for bitstr, count in res.items() if bitstr[-1] == '1' ] ) / np.sum(list(res.values()))
                elif len(idx_relevant) == 1: 
                    dummy_e, p0, p1 = expec_Z(res, idx_relevant[0], target_qubits=range(len(target_qubits)), ancilla_qubit=0)
                elif len(idx_relevant) == 2: 
                    dummy_e, p0, p1 = expec_ZZ(res, idx_relevant[0], idx_relevant[1], target_qubits=range(len(target_qubits)), ancilla_qubit=0)
                expval = p0 - p1
                Im_H_ij += Hamil_coeffs[idx_H] * expval
            #print("Im H_{ij}", Im_H_ij)

            H[it, j] = Re_H_ij + 1j * Im_H_ij
            H[j, it] = Re_H_ij - 1j * Im_H_ij

        # solve generalized eigenvalue problem
        Nsub = N[:it+1, :it+1]
        Hsub = H[:it+1, :it+1]        
        lam, v = scipy.linalg.eigh(Nsub) 
        # truncate orthogonal basis with small eigenvalues
        cols = [ i for i in range(it+1) if lam[i] >= 1.e-6]
        r = len(cols)
        Ur = v[:,cols]
        sq_Sigma_inv = np.diag(lam[cols]**(-0.5))

        X = Ur @ sq_Sigma_inv @ Ur.conj().T
        Xdag = X.conj().T
        tildeH = X @ Hsub @ Xdag
        w, v = scipy.linalg.eigh(tildeH)        
        w_r = w[-r:]
        ws += [w_r]

        print("eigs of N: ", lam, "cond", np.linalg.cond(Nsub), "r:", r)
        print(f"w: {w_r}")
        print("")
    return H[:it+1,:it+1], N[:it+1,:it+1], ws

In [133]:
# noisy simulator 
from qiskit_ibm_runtime.fake_provider import FakeQuebec
from qiskit_aer import AerSimulator

device_backend = FakeQuebec()
sim_quebec = AerSimulator.from_backend(device_backend)


In [134]:
def construct_X_and_Y(snapshots, d=8): # d is the number of delay
    N = len(snapshots)
    X = np.zeros((d, N-d), dtype=np.complex128)
    Y = np.zeros((d, N-d), dtype=np.complex128)
    for j in range(N-d):
        for i in range(d):
            idx = j + i 
            X[i,j] = snapshots[idx]
            Y[i,j] = snapshots[idx+1]
    return X, Y

def ODMD(delta_t, max_iterations, trotter_steps, num_shot, using_statevector, d=8, tol_SVD=1.e-8, verbose=False):
    ancilla_qubits=[0]
    target_qubits=list(range(1,Norb+1))
    Ntar = len(target_qubits)

    if Norb == 4 and Nocc == 2:
        Uprep = ansatz(params_exact, "FCI")
    else:
        Uprep = ansatz(dummy_params, "HF")

    cU0 = QuantumCircuit(Ntar)
    cU0.append(Uprep, range(Ntar))        
    cU0 = cU0.decompose(reps=4)
    # if using_noisy_simulator:
    #     cU0 = transpile(cU0, backend=backend, optimization_level=2)
    cU0 = cU0.to_gate().control(1)

    snapshots = np.zeros(max_iterations, dtype=np.complex128)
    snapshots[0] = 1.0
    for it in range(1, max_iterations):
        Uj = PauliEvolutionGate(hamiltonian_pauli_ops, it*delta_t, synthesis=SuzukiTrotter(order=1,reps=trotter_steps))
        gate_cUj = make_cU(Uprep, Uj, Ntar)
        overlap = measure_overlap(num_shot, Ntar, cU0, gate_cUj, ancilla_qubits, target_qubits, sampler, using_statevector)
        snapshots[it] = overlap
    print(f"Max iteration: {max_iterations:5d} trotter_steps: {trotter_steps:5d} delta_t: {delta_t:12.8f}")

    # Observable DMD
    if verbose:
        print("snapshots of <U0|Uj>:", snapshots)
    X, Y = construct_X_and_Y(snapshots)

    # SVD of X 
    U, Sigma, Vh = np.linalg.svd(X, full_matrices=False)
    if verbose:
        print("Sigma", Sigma)

    trank = np.sum(Sigma > tol_SVD)
    Ur = U[:,:trank]
    Sigmar = np.diag(Sigma[:trank])
    Vhr = Vh[:trank,:]

    # A =Y X^+ = Y (V Sigma^-1 Udag)
    A = Y @ Vhr.T.conj() @ np.linalg.inv(Sigmar) @ Ur.T.conj()

    # Check |AX - Y| 
    if verbose:
        print("Check |AX - Y|", np.linalg.norm(A @ X - Y))

    # Eigen values of A would be exp(-iE_j dt)
    lam, v = np.linalg.eig(A)
    #print("lam", lam)

    ## argument of eigenvalues
    arg_lam = np.angle(lam)  
    #print("angle[/pi]", arg_lam / (np.pi))
    print("E?", - (arg_lam) / delta_t)

    arg_in0to2pi = arg_lam % (2 * np.pi)
    idx_E0 = np.argmax(arg_in0to2pi)
    E0 =  - arg_lam[idx_E0] / delta_t

    idx_closer = np.argmin( np.abs( Egs_exact - ( - arg_lam / delta_t )))
    E0 = - arg_lam[idx_closer] / delta_t

    print(f"E0: {E0:12.8f} abs. diff.: {np.abs(E0 - Egs_exact):9.2e}")
    print("evals", evals)
    print("")
    return E0

if __name__ == "__main__":
    
    method_num = 2 # can be 0 (statevector), 1 (shot, noiseless), 2 (shot, noisy)

    if method_num == 0:
        num_shot = 1
        using_statevector = True
        using_noisy_simulator = False
    elif method_num == 1:
        using_statevector = False 
        num_shot=10**5
        using_noisy_simulator = False
    elif method_num == 2:
        using_statevector = False 
        num_shot=1024
        using_noisy_simulator = True

    if using_noisy_simulator:   
        device_backend = FakeQuebec()
        backend = AerSimulator.from_backend(device_backend)
        sim_quebec = AerSimulator.from_backend(device_backend)
        sampler = backend
    else:
        sampler = SamplerV2()

    dummy_params = [ ]

    Results = { }
    for delta_t in [0.1, 0.01, 0.001]:
        for trotter_steps in [1, 10, 100]:
            for max_iterations in [10, 100]:
                E0 = ODMD(delta_t, max_iterations, trotter_steps, num_shot, using_statevector, d=8, tol_SVD=1.e-8)
                Results[(delta_t, trotter_steps, max_iterations)] = E0
                
    print("delta_t, trotter_steps, max_iterations")
    for tmp_key, tmp_val in Results.items():
        delta_t, trotter_steps, max_iterations = tmp_key
        tmp_val = np.abs( tmp_val - Egs_exact)
        print("|" + str(delta_t) + "|" + str(trotter_steps) + "|" + str(max_iterations) + "|" + str("%9.1e" % tmp_val) + "|")



Max iteration:    10 trotter_steps:     1 delta_t:   0.10000000
E? [ 22.84599931  13.52463012  20.39776734   1.37146974 -30.68808621
  20.00002096  18.74122121  -5.0963347 ]
E0:   1.37146974 abs. diff.:  1.82e-01
evals [1.18985184 3.29649666 5.34       5.34       7.42853393 9.44511758]

Max iteration:   100 trotter_steps:     1 delta_t:   0.10000000
E? [ 17.35180332   8.94065147  -0.65224753  -6.89175358 -17.65867227
  27.93481778 -28.716938   -25.43113981]
E0:  -0.65224753 abs. diff.:  1.84e+00
evals [1.18985184 3.29649666 5.34       5.34       7.42853393 9.44511758]

Max iteration:    10 trotter_steps:    10 delta_t:   0.10000000
E? [ 10.25956065 -20.62237582 -20.97362598  23.68141078  12.79143152
  30.61826628 -19.10743499   9.86349823]
E0:   9.86349823 abs. diff.:  8.67e+00
evals [1.18985184 3.29649666 5.34       5.34       7.42853393 9.44511758]

Max iteration:   100 trotter_steps:    10 delta_t:   0.10000000
E? [ -8.7385857   -0.21956803   8.31202312 -18.10505515 -25.81528037
  2